In [1]:
import numpy as np
import tensorflow as tf
import tensornetwork as tn

In [2]:
class SimpleTNLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SimpleTNLayer, self).__init__()

        self.a = tf.Variable(tf.random.uniform(shape=(2, 10)), 
                                               name='a', 
                                               trainable=True)
        self.b = tf.Variable(tf.random.uniform(shape=(2, 10, 10, 2)), 
                                               name='b', 
                                               trainable=True)
        self.c = tf.Variable(tf.random.uniform(shape=(2, 10)), 
                                               name='c', 
                                               trainable=True)

    def infer_single(self, input):
        node_a = tn.Node(self.a, backend='tensorflow')
        node_b = tn.Node(self.b, backend='tensorflow')
        node_c = tn.Node(self.c, backend='tensorflow')

        input_a = tn.Node(input[0, :], backend='tensorflow')
        input_b = tn.Node(input[1, :], backend='tensorflow')
        input_c = tn.Node(input[2, :], backend='tensorflow')

        input_a[0] ^ node_a[0]
        input_b[0] ^ node_b[0]
        input_c[0] ^ node_c[0]
        node_a[1] ^ node_b[1]
        node_b[2] ^ node_c[1]
        
        final_node = tn.contractors.auto([node_a, node_b, node_c, input_a, input_b, input_c],
                                         output_edge_order=[node_b[3]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single, inputs)


In [3]:
tn_model = tf.keras.Sequential([
    tf.keras.Input((3, 2)),
    SimpleTNLayer(),
    tf.keras.layers.Softmax()
])

In [4]:
tn_model.trainable_variables

[<tf.Variable 'a:0' shape=(2, 10) dtype=float32, numpy=
 array([[0.80024827, 0.55311525, 0.0580951 , 0.78308165, 0.72722125,
         0.8616736 , 0.99308705, 0.18426919, 0.62930834, 0.53739023],
        [0.45606863, 0.07109189, 0.79261005, 0.8844588 , 0.28554714,
         0.83376586, 0.63506126, 0.671885  , 0.10235178, 0.7247834 ]],
       dtype=float32)>,
 <tf.Variable 'b:0' shape=(2, 10, 10, 2) dtype=float32, numpy=
 array([[[[0.59801304, 0.04137778],
          [0.34153318, 0.30735517],
          [0.7488799 , 0.00265205],
          [0.16239965, 0.01626849],
          [0.6469878 , 0.34007537],
          [0.43665183, 0.6307303 ],
          [0.79636073, 0.3516618 ],
          [0.4212643 , 0.8995645 ],
          [0.16244078, 0.5486909 ],
          [0.7423605 , 0.3652308 ]],
 
         [[0.752929  , 0.2860998 ],
          [0.39860582, 0.88444173],
          [0.7725911 , 0.07124674],
          [0.11541712, 0.34453022],
          [0.6528188 , 0.71554375],
          [0.44057775, 0.5232097 ],